<a href="https://colab.research.google.com/github/swaps95shah/DL_coursera/blob/master/ShahSwapnil_HW04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
"""
Author: Swapnil Shah
Email: sss4174@rit.edu
Acknowledgements and code references:
1. https://towardsdatascience.com/convolutional-neural-networks-for-beginners-practical-guide-with-python-and-keras-dc688ea90dca
2. https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/
"""

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!cp drive/'My Drive'/HW4_CV/fer3and4train.csv .

In [0]:
!cp drive/'My Drive'/HW4_CV/fer3and4test.csv .


In [0]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [0]:
"""
function from helper.py to get train and test data split into data and labels
"""
def getBinaryfer13Data(filename):
    Y = []
    X = []
    first = True
    for line in open(filename, 'rU'):
        if first:
            first = False
        else:
            row = line.split(',')
            y = int(row[0])
            type = row[2]
            if y == 3 or y == 4:	#3=happy; 4=sad
                Y.append(abs(y-4))  #we want to store 1 for happy, 0 for sad
                X.append([int(p) for p in row[1].split()])
    return np.array(X) / 255.0, np.array(Y)

In [78]:
from keras.utils import to_categorical
train_x, train_y = getBinaryfer13Data('fer3and4train.csv') #data label split for train
val_x, val_y = getBinaryfer13Data('fer3and4test.csv') #data label split for test
train_x = train_x.reshape(12066,48,48,1) #reshape to work with keras torch backend
val_x = val_x.reshape(3000,48,48,1)
train_y = to_categorical(train_y) #one hot encoding required since last layer has 2 neurons
val_y = to_categorical(val_y)
train_x.shape
train_y.shape


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: 'U' mode is deprecated
  


(12066, 2)

In [0]:
import keras
from keras import layers
from keras import models

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(48, 48, 1))) #conv layer
model.add(layers.MaxPooling2D((2, 2))) #reduce dimensions
model.add(layers.Conv2D(64, (5, 5), activation='relu')) #second conv layer
model.add(layers.MaxPooling2D((2, 2))) #reduce dimensions

In [0]:
model.add(layers.Flatten()) #flatten tensor
model.add(layers.Dense(2, activation='softmax')) #2 neurons with softmax activation

In [82]:
model.summary() #display model architecture

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 44, 44, 32)        832       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 18, 18, 64)        51264     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 9, 9, 64)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 5184)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 10370     
Total params: 62,466
Trainable params: 62,466
Non-trainable params: 0
__________________________________________________

In [83]:
batch_size = 100
epochs = 200

#categorical crossentropy used since we did one hot encoding
#binary crossentropy would be used for single neuron with sigmoid activation function
#optimizer is standard gradient descent
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1
          )

Epoch 1/200
12066/12066 [==============================] - 2s 187us/step - loss: 0.6913 - acc: 0.5336
Epoch 2/200
12066/12066 [==============================] - 2s 141us/step - loss: 0.6856 - acc: 0.5692
Epoch 3/200
12066/12066 [==============================] - 2s 140us/step - loss: 0.6780 - acc: 0.5995
Epoch 4/200
12066/12066 [==============================] - 2s 144us/step - loss: 0.6675 - acc: 0.6099
Epoch 5/200
12066/12066 [==============================] - 2s 143us/step - loss: 0.6537 - acc: 0.6267
Epoch 6/200
12066/12066 [==============================] - 2s 143us/step - loss: 0.6411 - acc: 0.6422
Epoch 7/200
12066/12066 [==============================] - 2s 143us/step - loss: 0.6292 - acc: 0.6479
Epoch 8/200
12066/12066 [==============================] - 2s 141us/step - loss: 0.6204 - acc: 0.6575
Epoch 9/200
12066/12066 [==============================] - 2s 143us/step - loss: 0.6042 - acc: 0.6795
Epoch 10/200
12066/12066 [==============================] - 2s 142us/step - loss: 

In [84]:
#run trained model on test data and print results
test_loss, test_acc = model.evaluate(val_x, val_y)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

3000/3000 [==============================] - 0s 162us/step
Test loss: 0.4931298967997233
Test accuracy: 0.8116666666666666
